In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv("../input/railway-data-imageslabels-incsv/Railway_data_images+labels/train.csv", index_col = 0)
train.head()

In [2]:
train.info()

In [3]:
train["labels"].unique()

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [7]:
 #Import label encoder
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
train['labels']= label_encoder.fit_transform(train['labels'])
 
train['labels'].unique()

In [8]:
list(label_encoder.inverse_transform([0]))

In [9]:
train["labels"].value_counts()

In [ ]:
import os 

train_image = []

#Y = []

data_dir = '../input/railway-data-imageslabels-incsv/Railway_data_images+labels'
image_list = train['images'].to_list()


for im_name in tqdm(image_list):
    path = os.path.join(data_dir, im_name)
    img = image.load_img(path, target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
    #Y.append(train['diseasetype'][i])

#Y = np.array(Y)
X = np.array(train_image)

In [ ]:
X.shape

In [ ]:
plt.imshow(X[570])

In [ ]:
import seaborn as sns
# visualize the target variable
g = sns.countplot(train['labels'])
g.set_xticklabels(['Defective','Non Defective'])
plt.show()

In [ ]:
Y = train['labels'].values

In [ ]:
Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.2, stratify = Y)

In [ ]:
Y_test

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
#from keras.applications import V2
#model = VGG16()
from  tensorflow.keras.models import Model
# from keras.layers import concatenate
from  tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, GRU, Reshape, Concatenate,Bidirectional, Conv2D, MaxPooling2D, UpSampling2D, Flatten
#from  tensorflow.keras.applications.mobilenet import preprocess_input

from  tensorflow.keras.layers import GaussianNoise
input_img = Input(shape = (400,400,3))
#f1_x = tf.keras.layers.Flatten()
#encoder
f1_x = Conv2D(128, (6, 6), activation='relu', kernel_initializer='he_normal')(input_img) #28 x 28 x 32
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.4)
f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) #14 x 14 x 32
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.35)
f1_x = Conv2D(64, (4, 4), activation='relu', kernel_initializer='he_normal', padding='valid')(f1_x) #14 x 14 x 64
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.33)
f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) #7 x 7 x 64
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.3)
f1_x = Conv2D(64, (4, 4), activation='relu', padding='valid',kernel_initializer='he_normal')(f1_x) #7 x 7 x 128 (small and thick)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.4)
f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.49)
#decoder
f1_x = Conv2D(64, (6, 6), activation='relu', padding='valid',kernel_initializer='he_normal')(f1_x) #7 x 7 x 128
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.36)
f1_x = UpSampling2D((2,2))(f1_x) # 14 x 14 x 128
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.38)
f1_x = Conv2D(128, (6, 6), activation='relu', padding='valid',kernel_initializer='he_normal')(f1_x) # 14 x 14 x 64
tf.keras.layers.BatchNormalization()
#dropout=0.2
f1_x = UpSampling2D((2,2))(f1_x) # 28 x 28 x 64
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.37)
f1_x = Conv2D(1, (6, 6), activation='sigmoid', padding='valid', kernel_initializer='he_normal')(f1_x) # 28 x 28 x 1
tf.keras.layers.BatchNormalization()
tf.keras.layers.BatchNormalization()

tf.keras.layers.Dropout(rate=0.5)
#f1_x = MinPooling2D(pool_size=(3, 3))(f1_x)     
# f1_base = Xception(weights='imagenet', include_top=False, input_shape=(450,450,3))
# f1_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3))
f1_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(400,400,3))
#tf.keras.layers.Flatten()
tf.keras.layers.Dropout(rate=0.45)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.5)
tf.keras.layers.BatchNormalization()
f1_x = f1_base.output
tf.keras.layers.BatchNormalization()
f1_x = GlobalAveragePooling2D()(f1_x)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.4)
#f1_x = Conv2D(128, (3, 3), activation='relu', padding='same')(f1_x) #7 x 7 x 128
#f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) 
f1_x = Reshape([1,1536])(f1_x)
#tf.keras.layers.Flatten()
f1_x = Bidirectional(tf.keras.layers.LSTM(2900, 
return_sequences=False, 
dropout=0.55),
input_shape=[1,1024],
merge_mode='concat')(f1_x)
tf.keras.layers.BatchNormalization()
#f1_x = Conv2D(128, (3, 3), activation='relu', padding='same')(f1_x) #7 x 7 x 128
#f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) 
#Regularization with noise
f1_x = GaussianNoise(0.20)(f1_x)
tf.keras.layers.BatchNormalization()
f1_x = Dense(100,activation='relu', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l1_l2(0.01),kernel_constraint=tf.keras.constraints.max_norm(1.))(f1_x)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.5)
tf.keras.layers.BatchNormalization()
f1_x = Dense(100,activation='relu', kernel_initializer='he_normal',kernel_constraint=tf.keras.constraints.max_norm(1.))(f1_x)
tf.keras.layers.Dropout(rate=0.58)
tf.keras.layers.BatchNormalization()
#tf.keras.layers.BatchNormalization()
#tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.46)
tf.keras.layers.BatchNormalization()
#he_avg_int = keras.initializers.VarianceScaling(scale=2, mode = 'fan_avg', distribution='uniform')
#f1_x = Conv2D(64, (3, 3), activation='relu', padding='same')(f1_x)
#tf.keras.layers.Flatten()
f1_x = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(f1_x)
model_1 = Model(inputs=[f1_base.input],outputs=[f1_x])
print(model_1.summary())

In [ ]:
model_1.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
Y_test.shape

In [ ]:
result = model_1.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test), batch_size=8, verbose=1)

In [ ]:
import numpy as np
print(result.history['val_accuracy'])
print(np.mean(result.history['val_accuracy']))
print(np.max(result.history['val_accuracy']))
print(result.history['accuracy'])
print(np.mean(result.history['accuracy']))
print(np.max(result.history['accuracy']))

In [ ]:
import matplotlib.pyplot as plt
history = result
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('training and validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model_1.evaluate(X_test,Y_test,batch_size=6)

In [ ]:
y_pred=model_1.predict(X_test)

In [ ]:
Y_test

In [ ]:
#predict
y_pred=model_1.predict(X_test) # (n_images, 1)
y_classes = ((y_pred > 0.5)+0).ravel() 
#y_pred=np.argmax(y_pred,axis=1)
y_pred

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 10))
for ax, image, prediction in zip(axes, X_test, y_classes):
    ax.set_axis_off()
    #image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title(f"Prediction: {prediction}")

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_classes,Y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(Y_test, y_classes)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np
from sklearn import datasets, svm, metrics

In [ ]:
#get classification report
print(classification_report(y_classes,Y_test))

In [ ]:
cm = print(confusion_matrix(y_classes, Y_test))